# Project

In [1]:
import pandas as pd
import json as JSON
import numpy as np

from collections import Counter
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

from pyspark.ml.fpm import PrefixSpan
from pyspark.shell import sc
from pyspark.sql.functions import desc
from pyspark.sql.types import Row


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.3.0
      /_/

Using Python version 3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022 16:36:42)
Spark context Web UI available at http://10.0.13.166:4040
Spark context available as 'sc' (master = local[*], app id = local-1665916615341).
SparkSession available as 'spark'.


In [2]:
df = pd.read_csv('keystrokes-recipes.csv')
KEYWORDS = ['Shift', 'Backspace', 'Enter', 'ArrowDown',
            'ArrowLeft', 'ArrowRight', 'ArrowUp', 'End', 'Control', 'CapsLock']

#create a copy of the dataset to another csv file

df.to_csv('keystrokes-recipes-modified.csv', index=False)

- ```keystrokes-recipes.csv``` is the original data and we keep it in case we want to look back at one moment
- ```keystrokes-recipes-modified.csv``` is the modified data


## Data cleaning and sorting


Our data consists of a csv file with event dates, user ids, keystrokes and the recipes they wrote.
We clean all the data by working throught the keystrokes first.

* We group the characters into the word written and separate between important keywords typed such as backspace, shift, enter etc. The sequence ['shift', 'p', 'e', 'r'] becomes ['shift', 'per'] 
* We sort by user id to get a better idea of every recipe every student has written.

## Processing the data


The first thing we did was isolate the keystrokes to a new ```json``` file saved in ```data/all_keystrokes.json```

The next step is to group words together and separate them from keywords and we work between each whitespace.
 
So for example this entry: 

```{'time': 1662252404346, 'character': 'Shift'}, {'time': 1662252404376, 'character': 'f'}, {'time': 1662252404505, 'character': 'i'}``` 

gives the following output: 

```{'time': 1662252404346, 'word': 'Shift'}, {'time': 1662252404505, 'word': 'fi'}```


In [3]:
def find_seq(chars):
    return ''.join(list(filter(lambda _ : _ not in KEYWORDS, chars)))

def separate_entry(json_values):
    new_data = []
    last_whitespace = 0
    characters = [arr[1] for arr in json_values]

    for i, (time, character) in enumerate(json_values):
        if character.isspace():
            word = characters[last_whitespace: i]
            if not any(i in word for i in KEYWORDS):
                new_data.append({'time': time, 'word': ''.join(word)})
            else:
                new_data.append({'time': time, 'word': find_seq(word)})
            last_whitespace = i+1

        elif character in KEYWORDS:
            new_data.append({'time': time, 'word': character})
        
        elif not character:
            continue

    return new_data

def compute():
    new_df = pd.DataFrame(pd.read_json('data/all_keystrokes.json'))
    arr = []
    for jsonf in new_df.values:
        sub_arr = []
        for d in jsonf:
            if d is not None:
                sub_arr.append([d['time'], d['character']])
        arr.append(sub_arr)

    result = []
    for jsonf in arr:
        result.append(separate_entry(jsonf))

    with open('data/new_data.json', 'w') as f:
        JSON.dump(result, f)
    return

compute()



Basically, ``` compute()``` will format the data for the ```separate_entry``` function and when everything is computed, it dumps all the data in a new json file.

```separate_entry``` computes the words between each space character, all the while separating words from keywords. It uses the function ```find_seq``` to separate the characters from keywords so it allows to isolate words between each whitespaces.


## Modifying the CSV file

We just modify the keystroke data for each row of the original data in ```keystrokes-recipes.csv``` but apply it to ```keystrokes-recipes-modified.csv```

In [4]:
jsons = pd.read_json('data/new_data.json').values.tolist()

for i, json in enumerate(jsons):
    jsons[i]= list(filter(lambda _ : _ is not None, json))

def write_to_csv_file(filename, recipes_len):
    df = pd.read_csv(filename)
    for i in range(recipes_len):
        df.loc[i,'ks'] = jsons[i]
    df.to_csv(filename, index=False)

write_to_csv_file('keystrokes-recipes-modified.csv', len(jsons))


## Sorting by user id

We want to sort by user id in order to differentiate behaviour between different people more easily

In [5]:
csv_filename = 'keystrokes-recipes-modified.csv'
pd.read_csv(csv_filename).sort_values(by='user_id', ascending=True).to_csv(csv_filename, index=False)

## Identifying patterns between different student recipes

Count the most common words used in the recipes and removing stopwords (they're noisy)

In [6]:
data = pd.read_csv(csv_filename)['recipe'].values
stop_words = set(stopwords.words('english'))
recipes_copy = list(map(lambda _ : word_tokenize(_), data.copy()))
recipes_without_stopwords = []
for recipe in recipes_copy:
    recipes_without_stopwords.append([w for w in recipe if not w.lower() in stop_words])    

flat_list = [item for sublist in recipes_without_stopwords for item in sublist]
c = Counter(flat_list)
print(c.most_common(100))


[('.', 15092), (',', 7062), ('-', 3485), ('1', 2802), (')', 2406), ('2', 2135), ('add', 1905), ('minutes', 1834), ('pan', 1765), (':', 1731), ('Add', 1550), ('3', 1298), ('water', 1253), ('oil', 1231), ('4', 1207), ('heat', 1180), ('5', 1077), ('garlic', 967), ('(', 961), ('salt', 958), ('sauce', 916), ('pepper', 886), ('chicken', 820), ('cook', 783), ('6', 752), ('butter', 744), ('onion', 740), ('oven', 726), ('rice', 725), ('bowl', 717), ('Ingredients', 717), ('cheese', 694), ('large', 681), ('mix', 649), ('pasta', 613), ('cup', 604), ('10', 575), ('place', 564), ('7', 537), ('medium', 537), ('mixture', 535), ('stir', 516), ('flour', 500), ('8', 496), ('cooked', 489), ('top', 480), ('sugar', 453), ('chopped', 450), ('Place', 443), ('Step', 434), ('tomatoes', 432), ('onions', 420), ('boil', 419), ('olive', 418), ('potatoes', 417), ('one', 402), ('cream', 396), ('tbsp', 395), ('eggs', 392), ('powder', 387), ('brown', 386), ('small', 385), ('cooking', 375), ('tsp', 373), ('pot', 367), (

Most of the recipes contain an order in which to proceed - ie 1) --- 2) --- 
Also interesting to see most of the common words are not ingredients

### Pattern mining

There are a total of 1091 recipes from 187 students. What we would like to do is either apply pattern mining for each student and their recipes to better understand each users writing. Or, we pick one recipe from each student and extract the underlying patterns each student share.

Given our data, we want to extract the longest common patterns as the shorter ones will only consist of heavily used words such as 'the', 'and' ... 
Hence, given the running time of running the ```PrefixSpan``` algorithm using ```pyspark``` to find the patterns with ```maxPatternLength=10``` is over 10 minutes for only 5 recipes, it would take approximately 1870 minutes (1091 recipes and 187 students gives roughly 5 recipes per student) to compute patterns for each student recipes. So, we ought to only pick one from each student and apply the ```PrefixSpan``` algorithm. Regardless - applying prefix span on the written recipes results in unsatisfying output.

To reduce the amount of computation of the algorithm, we simply pick the first recipe each student writes

In [7]:
sorted_df = pd.read_csv(csv_filename)
first_recipe_each_student = sorted_df.groupby('user_id').head(1)
one_recipe_per_student = []
for index in first_recipe_each_student.index:
    one_recipe_per_student.append(word_tokenize((sorted_df.loc[index]['recipe'])))

From, there we can try to apply the PS algorithm

In [8]:
#restricting to 5 recipes for now - the running time is too large
recipes = one_recipe_per_student[:5]

copy = recipes.copy()

range_ = range(len(copy))
for i in range_:
    copy[i] = Row(sequence=[recipes[i]])

l = [copy[i] for i in range_]

df = sc.parallelize(l).toDF()

prefixSpan = PrefixSpan(minSupport=0.7, maxPatternLength=8, maxLocalProjDBSize=32000000)

# Find frequent sequential patterns.
prefixSpan.findFrequentSequentialPatterns(df).sort(desc('freq')).show(15,False)


+----------------------+----+
|sequence              |freq|
+----------------------+----+
|[[and, and, and, and]]|5   |
|[[of]]                |5   |
|[[and, and]]          |5   |
|[[3]]                 |5   |
|[[1]]                 |5   |
|[[5]]                 |5   |
|[[with]]              |5   |
|[[the]]               |5   |
|[[.]]                 |5   |
|[[in]]                |5   |
|[[4]]                 |5   |
|[[,]]                 |5   |
|[[a]]                 |5   |
|[[and]]               |5   |
|[[2]]                 |5   |
+----------------------+----+
only showing top 15 rows



Can't really make anything of this which makes sense bc it looks for sequential patterns - not the general idea.

What we can do now is try to map each typed character as an insert action, count each backspace as a delete action and from those sequences, generate some patterns under the form (insert - insert - delete - insert ...)

We can also try to check if any of the commands typed are of the form ```CTRL + C``` - ```CTRL + V``` to see if any of the students might have cheated

* pending approval from thiemo 